In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import copy
import json

# Import data

In [7]:
borough_coordinates = gpd.read_file("../data/input/misc/borough_coordinates.json").set_index('borough')

In [8]:
# Import Borough json
with open('../data/raw/misc/london_boroughs.json') as json_file:
    boroughs_coords = json.load(json_file)

In [9]:
# Import commuter data
borough_commuters = pd.read_csv("../data/raw/commuter/commuting-patterns-borough.csv")
bc = copy.deepcopy(borough_commuters)

# Clean Borough names and Origin/Destination dataframe

In [10]:
# Collect borough names
boroughs = []
boroughs_no_spaces = []
for f in tqdm(boroughs_coords['features']):
    boroughs.append(f['properties']['name'])
    boroughs_no_spaces.append(f['properties']['name'].replace(" ", ""))

100%|██████████| 33/33 [00:00<00:00, 48685.20it/s]


In [11]:
# Subset columns based on borough names
bc = bc[['Origin Area']+boroughs+['Into area (A)','Out of area (B)']]

# Subset rows based on borough names
bc = bc[bc['Origin Area'].isin(boroughs)]

In [12]:
# Remove spaces from origin names
bc.loc[:,'Origin Area'] = bc['Origin Area'].apply(lambda x: x.replace(" ",""))
# Remove spaces from destination names
bc = bc.rename(columns = dict(zip(np.sort(boroughs), np.sort(boroughs_no_spaces))))

In [13]:
# Separate inflow/outflow columns from rest of O/D matrix
in_out_flows = copy.deepcopy(bc[['Origin Area','Into area (A)','Out of area (B)']])
od_matrix = copy.deepcopy(bc.drop(columns=['Into area (A)','Out of area (B)']))

# Rename columns
in_out_flows = in_out_flows.rename(columns={'Origin Area':'Origin','Into area (A)':'Inflows','Out of area (B)':'Outflows'})
od_matrix = od_matrix.rename(columns={'Origin Area':'Origin'})

# Update index
in_out_flows = in_out_flows.set_index(keys='Origin')
od_matrix = od_matrix.set_index(keys='Origin')

# Sort by index
in_out_flows = in_out_flows.sort_index(axis=0)
od_matrix = od_matrix.sort_index(axis=1).sort_index(axis=0)

In [14]:
# Get origin supply
origin_supply = copy.deepcopy(od_matrix.sum(axis=1).reset_index())
# origin_supply = origin_supply.rename(columns={'index':'Origin',0:'Supply'})
origin_supply.columns = ['Origin','Supply']
origin_supply = origin_supply.set_index('Origin').sort_index()
origin_supply['lon'] = borough_coordinates['lon'].values
origin_supply['lat'] = borough_coordinates['lat'].values

In [15]:
# Get destination demand
destination_demand = copy.deepcopy(od_matrix.sum(axis=0).reset_index())
#destination_demand = destination_demand.rename(columns={'Origin':'Destination',0:'Demand'})
destination_demand.columns = ['Destination','Demand']
destination_demand = destination_demand.set_index('Destination').sort_index()
destination_demand['lon'] = borough_coordinates['lon'].values
destination_demand['lat'] = borough_coordinates['lat'].values

In [16]:
# Replace zeros with ones
od_matrix = od_matrix.replace(0,1)

In [17]:
od_matrix

,BarkingandDagenham,Barnet,Bexley,Brent,Bromley,Camden,CityofLondon,Croydon,Ealing,Enfield,...,Merton,Newham,Redbridge,RichmonduponThames,Southwark,Sutton,TowerHamlets,WalthamForest,Wandsworth,Westminster
Origin,,,,,,,,,,,,,,,,,,,,,
BarkingandDagenham,20434,194,96,178,66,1500,3641,104,188,410,...,67,4460,5940,28,926,38,4069,1047,165,3280
Barnet,96,44045,34,5467,76,12080,7709,148,1573,4098,...,146,387,305,229,1792,44,2511,555,536,16330
Bexley,362,132,33681,144,4998,2470,6580,710,188,123,...,190,574,111,90,4666,170,2825,222,618,7692
Brent,40,6124,28,32105,66,8105,4145,187,6703,426,...,203,279,116,396,1565,56,1655,210,669,16418
Bromley,134,162,3199,201,51289,3780,9855,6268,293,84,...,647,419,100,191,6057,796,3385,196,1371,12802
Camden,36,1496,32,1350,60,26535,8795,147,643,295,...,139,290,84,195,1784,54,2614,204,588,18829
CityofLondon,6,14,1,16,1,335,1627,3,9,6,...,6,16,1,6,103,3,266,16,16,602
Croydon,85,204,300,329,5152,3248,5925,65033,405,120,...,3521,202,64,480,4513,6744,2000,130,4270,10583
Ealing,30,967,33,5263,91,4547,4855,182,42004,223,...,361,229,79,1578,1328,65,1420,127,944,13967


# Export data as dataframe and numpy array

In [38]:
# Export to csv
od_matrix.to_csv('../data/validation/commuter/borough_od_matrix.csv')
# Export to txt
np.savetxt('../data/validation/commuter/borough_od_matrix.txt',od_matrix.to_numpy())

In [39]:
# Export to csv
origin_supply.to_csv('../data/validation/commuter/borough_origin_supply.csv')
# Export to txt
np.savetxt('../data/validation/commuter/borough_origin_supply.txt',origin_supply['Supply'].to_numpy())
np.savetxt('../data/validation/commuter/borough_origin_locations.txt',origin_supply[['lon','lat']].to_numpy())

In [41]:
# Export to csv
destination_demand.to_csv('../data/validation/commuter/borough_destination_demand.csv')
# Export to txt
np.savetxt('../data/validation/commuter/borough_destination_demand.txt',destination_demand['Demand'].to_numpy())
np.savetxt('../data/validation/commuter/borough_locations.txt',destination_demand[['lon','lat']].to_numpy())

In [37]:
np.savetxt('../data/input/commuter/destination_locations.txt',destination_demand[['lon','lat']].to_numpy())